In [ ]:
# Bajar datos

# Varios trimestres
# Existen version en .csv, .dbf, .dta. .sav

#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2025_trim2_dta.zip # 2025_2
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2023_trim4_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim4_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim1_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim4_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim3_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2020trim1_dbf.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2019trim4_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2015trim1_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2010trim1_dbf.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim2_dbf.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim1_dbf.zip

########
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2025_trim2_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim4_dta.zip

####
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2020_trim3_dbf.zip




# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim2_dbf.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim4_dbf.zip


# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2023_trim1_dbf.zip


In [ ]:
import os
import requests
import zipfile
import time

# --- Configuración de Reintento ---
MAX_RETRIES = 3
RETRY_DELAY = 5 # Segundos

def get_url(year, quarter, file_format):
    """
    Construye la URL de descarga para un año, trimestre y formato dados,
    manejando los diferentes patrones de nomenclatura del INEGI.
    
    Args:
        year (int): El año deseado.
        quarter (int): El trimestre deseado (1-4).
        file_format (str): 'dta' o 'dbf'.
        
    Returns:
        str: La URL completa para la descarga, o None si no se encuentra un patrón.
    """
    base_url = "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/"
    filename = ""

    # --- 1. Formato Antiguo (2005 T1 a 2019 T4) ---
    # Estructura: AAAAtrimT_formato.zip
    if 2005 <= year <= 2019:
        filename = f"{year}trim{quarter}_{file_format}.zip"

    # --- 2. Formato de Transición con "_n_" (2020 T3 a 2022 T4) ---
    # Estructura: enoe_n_AAAA_trimT_formato.zip
    elif (year == 2020 and quarter >= 3) or (2021 <= year <= 2022):
        filename = f"enoe_n_{year}_trim{quarter}_{file_format}.zip"
    
    # --- 3. Formato Moderno (2023 en adelante) ---
    # Estructura: enoe_AAAA_trimT_formato.zip
    elif year >= 2023:
        filename = f"enoe_{year}_trim{quarter}_{file_format}.zip"

    if filename:
        return base_url + filename
    
    return None

def download_and_extract_data(start_year, start_quarter, end_year, end_quarter, file_format):
    """
    Descarga y organiza las bases de datos de la ENOE dentro del rango y formato especificados.
    """
    
    # Directorios base dinámicos según el formato
    temp_dir = "data/temporal"
    main_data_dir = f"data/ENOE_{file_format}"
    
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(main_data_dir, exist_ok=True)
    
    print(f"\nLos archivos se guardarán en: '{main_data_dir}'")
    
    current_year = start_year
    
    # Generar la secuencia de trimestres a iterar
    while True:
        for current_quarter in range(1, 5):
            
            # 1. Comprobar si está en el rango de inicio
            if current_year < start_year or \
               (current_year == start_year and current_quarter < start_quarter):
                continue
            
            # 2. Comprobar si está fuera del rango de fin
            if current_year > end_year or \
               (current_year == end_year and current_quarter > end_quarter):
                print("\n--- ¡Descarga completa! Se alcanzó el trimestre final. ---")
                return

            # 3. Manejar casos especiales sin datos
            if (current_year == 2020 and current_quarter in [1, 2]):
                print(f"⏩ Saltando {current_year} T{current_quarter}: No hay datos reportados por INEGI.")
                continue

            # 4. Obtener URL y ruta
            url = get_url(current_year, current_quarter, file_format)
            if url is None:
                print(f"⚠️ No se encontró URL válida para {current_year} T{current_quarter} con formato '{file_format}'. Se saltará este periodo.")
                continue

            file_name = url.split('/')[-1]
            temp_file_path = os.path.join(temp_dir, file_name)
            
            # Crear directorio de destino específico para el periodo
            dest_dir_name = f"ENOE_{current_year}_{current_quarter}"
            dest_dir = os.path.join(main_data_dir, dest_dir_name)
            
            # Si el directorio ya existe y no está vacío, saltar la descarga
            if os.path.exists(dest_dir) and os.listdir(dest_dir):
                print(f"✅ Directorio '{dest_dir_name}' ya existe y no está vacío. Saltando descarga.")
                continue

            os.makedirs(dest_dir, exist_ok=True)
            
            # 5. Descarga y reintento
            retries = 0
            download_successful = False
            
            while retries < MAX_RETRIES:
                print(f"\n⬇️ Intentando descargar {file_name} ({retries + 1}/{MAX_RETRIES})...")
                
                try:
                    with requests.get(url, stream=True, timeout=30) as r:
                        r.raise_for_status() # Lanza HTTPError para códigos 4xx/5xx
                        total_size = int(r.headers.get('content-length', 0))
                        
                        with open(temp_file_path, 'wb') as f:
                            # Simple indicador de progreso
                            downloaded = 0
                            for chunk in r.iter_content(chunk_size=8192):
                                f.write(chunk)
                                downloaded += len(chunk)
                                # Imprimir progreso en la misma línea
                                print(f"\r   Descargando... {downloaded / (1024*1024):.2f} MB / {total_size / (1024*1024):.2f} MB", end="")
                    
                    print() # Nueva línea después de la descarga
                    
                    if os.path.getsize(temp_file_path) > 0:
                        download_successful = True
                        break
                    else:
                        raise IOError("El archivo se descargó pero tiene un tamaño de 0 bytes.")
                        
                except (requests.exceptions.RequestException, IOError, Exception) as e:
                    retries += 1
                    print(f"\n❌ Error al descargar o verificar el archivo: {e}")
                    if retries < MAX_RETRIES:
                        print(f"🔄 Reintentando en {RETRY_DELAY} segundos...")
                        time.sleep(RETRY_DELAY)
                    else:
                        print(f"🚫 Falló la descarga de {file_name} después de {MAX_RETRIES} intentos.")
                        break

            # 6. Descompresión
            if download_successful:
                try:
                    with zipfile.ZipFile(temp_file_path, 'r') as zip_ref:
                        zip_ref.extractall(dest_dir)
                        print(f"✅ Éxito: {file_name} descomprimido en {dest_dir}")
                    
                    os.remove(temp_file_path)
                    
                except zipfile.BadZipFile:
                    print(f"⚠️ El archivo {file_name} está corrupto. Se mantiene en '{temp_dir}' para revisión.")
                except Exception as e:
                    print(f"⚠️ Error desconocido al descomprimir: {e}")

        current_year += 1
        
        # Límite de seguridad
        if current_year > 2026: 
            print("\n--- ¡Límite de año alcanzado! Terminando el proceso. ---")
            break

# --- Función Principal e Interacción con el Usuario ---
if __name__ == "__main__":
    
    def get_user_input():
        default_start_year = 2005
        default_start_quarter = 1
        # El año final por defecto es el año actual
        default_end_year = time.localtime().tm_year
        default_end_quarter = 4

        # 1. Selección de Formato
        while True:
            file_format_input = input("¿Qué formato de archivo deseas descargar? (dta/dbf) [Default: dta]: ").lower()
            if file_format_input in ['dta', 'dbf']:
                file_format = file_format_input
                break
            elif not file_format_input:
                file_format = 'dta'
                break
            else:
                print("Entrada inválida. Por favor, escribe 'dta' o 'dbf'.")
        
        print(f"Has seleccionado el formato: '{file_format}'")
        print("\n--- Configuración de Rango de Descarga (Formato AAAA T) ---")
        
        # 2. Rango de Fechas (el resto de tu lógica de input es excelente y se mantiene)
        start_y = int(input(f"Año de inicio (ej. 2005) [Default: {default_start_year}]: ") or default_start_year)
        start_q = int(input(f"Trimestre de inicio (1-4) [Default: {default_start_quarter}]: ") or default_start_quarter)
        end_y = int(input(f"Año de fin (ej. {default_end_year}) [Default: {default_end_year}]: ") or default_end_year)
        end_q = int(input(f"Trimestre de fin (1-4) [Default: {default_end_quarter}]: ") or default_end_quarter)

        return start_y, start_q, end_y, end_q, file_format

    # Obtener configuración y ejecutar
    s_y, s_q, e_y, e_q, f_format = get_user_input()
    print(f"\nComenzando la descarga desde {s_y} T{s_q} hasta {e_y} T{e_q}...")
    
    download_and_extract_data(s_y, s_q, e_y, e_q, f_format)

In [ ]:
# Esto es para .dbf

import os
import requests
import zipfile
import time

# --- Configuración de Reintento ---
MAX_RETRIES = 3
RETRY_DELAY = 5 # Segundos

def get_url(year, quarter):
    """
    Construye la URL de descarga para un año y trimestre dados,
    manejando los diferentes formatos del INEGI.
    """
    base_url = "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/"

    # --- 1. Formato Antiguo (2005 T1 a 2019 T4) ---
    # Estructura: AAAAtrimT_dbf.zip
    if 2005 <= year <= 2019:
        filename = f"{year}trim{quarter}_dbf.zip"
        return base_url + filename

    # --- 2. Formato de Transición/Especial (2020 T3 a 2021 T1) ---
    # Estructura: enoe_n_AAAA_trimT_dbf.zip
    if (year == 2020 and quarter >= 3) or (year == 2021 and quarter == 1):
        filename = f"enoe_n_{year}_trim{quarter}_dbf.zip"
        return base_url + filename

    # --- 3. Formato Moderno .DTA (2021 T2 a 2022 T4) ---
    # Estructura: enoe_n_AAAA_trimT_dta.zip
    if (year == 2021 and quarter >= 2) or (year == 2022 and 1 <= quarter <= 4):
        filename = f"enoe_n_{year}_trim{quarter}_dta.zip"
        return base_url + filename

    # --- 4. Formato Post-2023 (SIN '_n', Prueba .DBF y .DTA) ---
    # Estructura: enoe_AAAA_trimT_dbf.zip
    if year >= 2023:
        # 4a. Probar con la extensión .dbf
        filename_dbf = f"enoe_{year}_trim{quarter}_dbf.zip"
        url_dbf = base_url + filename_dbf
        
        try:
            if requests.head(url_dbf, timeout=5).status_code == 200:
                return url_dbf
        except requests.exceptions.RequestException:
            pass 
        
        # 4b. Probar con la extensión .dta (por si INEGI usa este formato sin '_n' en el futuro)
        filename_dta = f"enoe_{year}_trim{quarter}_dta.zip"
        url_dta = base_url + filename_dta
        
        try:
            if requests.head(url_dta, timeout=5).status_code == 200:
                return url_dta
        except requests.exceptions.RequestException:
            pass 

    return None

def download_and_extract_data(start_year, start_quarter, end_year, end_quarter):
    """
    Descarga y organiza las bases de datos de la ENOE dentro del rango especificado.
    """
    
    # Directorios base
    temp_dir = "data/temporal"
    main_data_dir = "data/ENOE"
    
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(main_data_dir, exist_ok=True)
    
    current_year = start_year
    
    # Generar la secuencia de trimestres a iterar
    while True:
        for current_quarter in range(1, 5):
            
            # 1. Comprobar si está en el rango de inicio
            if current_year < start_year or \
               (current_year == start_year and current_quarter < start_quarter):
                continue
            
            # 2. Comprobar si está fuera del rango de fin
            if current_year > end_year or \
               (current_year == end_year and current_quarter > end_quarter):
                print("\n--- ¡Descarga completa! Se alcanzó el trimestre final. ---")
                return

            # 3. Manejar casos especiales sin datos
            if (current_year == 2020 and current_quarter in [1, 2]):
                print(f"⏩ Saltando {current_year} T{current_quarter}: No hay datos reportados.")
                continue

            # 4. Obtener URL y ruta
            url = get_url(current_year, current_quarter)
            if url is None:
                print(f"⚠️ No se encontró URL válida para {current_year} T{current_quarter}. Se saltará este periodo.")
                continue

            file_name = url.split('/')[-1]
            temp_file_path = os.path.join(temp_dir, file_name)
            dest_dir = os.path.join(main_data_dir, f"ENOE{current_year}{current_quarter}")
            os.makedirs(dest_dir, exist_ok=True)
            
            # 5. Descarga y reintento
            retries = 0
            download_successful = False
            
            while retries < MAX_RETRIES:
                print(f"\n⬇️ Intentando descargar {file_name} ({retries + 1}/{MAX_RETRIES})...")
                
                try:
                    # Descarga el archivo
                    with requests.get(url, stream=True, timeout=30) as r:
                        r.raise_for_status() # Lanza HTTPError para códigos 4xx/5xx
                        with open(temp_file_path, 'wb') as f:
                            for chunk in r.iter_content(chunk_size=8192):
                                f.write(chunk)
                    
                    # Verifica si el archivo se descargó y tiene un tamaño > 0
                    if os.path.getsize(temp_file_path) > 0:
                        download_successful = True
                        break
                    else:
                        raise IOError("El archivo se descargó pero tiene un tamaño de 0 bytes.")
                        
                except (requests.exceptions.RequestException, IOError, zipfile.BadZipFile, Exception) as e:
                    retries += 1
                    print(f"❌ Error al descargar o verificar el archivo: {e}")
                    if retries < MAX_RETRIES:
                        print(f"🔄 Reintentando en {RETRY_DELAY} segundos...")
                        time.sleep(RETRY_DELAY)
                    else:
                        print(f"🚫 Falló la descarga de {file_name} después de {MAX_RETRIES} intentos.")
                        break

            # 6. Descompresión (solo si la descarga fue exitosa)
            if download_successful:
                try:
                    with zipfile.ZipFile(temp_file_path, 'r') as zip_ref:
                        zip_ref.extractall(dest_dir)
                        print(f"✅ Éxito: {file_name} descomprimido en {dest_dir}")
                    
                    # Limpiar el archivo .zip de la carpeta temporal después de la extracción
                    os.remove(temp_file_path)
                    
                except zipfile.BadZipFile:
                    print(f"⚠️ El archivo {file_name} está corrupto o no es un archivo zip válido. Se mantiene el archivo en temporal para revisión.")
                except Exception as e:
                    print(f"⚠️ Error desconocido al descomprimir: {e}")


        # Avanzar al siguiente año al terminar los trimestres
        current_year += 1
        
        # Establecer un límite de seguridad
        if current_year > 2025: 
            print("\n--- ¡Límite de año alcanzado! Terminando el proceso. ---")
            break

# --- Función Principal e Interacción con el Usuario ---
if __name__ == "__main__":
    
    def get_user_input():
        # Valores por defecto para el inicio y fin. Ajustar END_YEAR si es necesario.
        default_start_year = 2005
        default_start_quarter = 1
        default_end_year = 2025 
        default_end_quarter = 4

        print("--- Configuración de Rango de Descarga (Formato AAAA T) ---")
        
        # Obtener año de inicio
        while True:
            try:
                sy_input = input(f"Año de inicio (ej. 2005) [Default: {default_start_year}]: ")
                start_y = int(sy_input) if sy_input else default_start_year
                if 2005 <= start_y <= default_end_year:
                    break
                else:
                    print(f"Por favor, introduce un año entre 2005 y {default_end_year}.")
            except ValueError:
                print("Entrada inválida. Por favor, introduce un número.")

        # Obtener trimestre de inicio
        while True:
            try:
                sq_input = input(f"Trimestre de inicio (1-4) [Default: {default_start_quarter}]: ")
                start_q = int(sq_input) if sq_input else default_start_quarter
                if 1 <= start_q <= 4:
                    break
                else:
                    print("El trimestre debe ser un número entre 1 y 4.")
            except ValueError:
                print("Entrada inválida. Por favor, introduce un número.")

        # Obtener año de fin
        while True:
            try:
                ey_input = input(f"Año de fin (ej. 2025) [Default: {default_end_year}]: ")
                end_y = int(ey_input) if ey_input else default_end_year
                if start_y <= end_y <= default_end_year:
                    break
                else:
                    print(f"El año de fin debe ser mayor o igual al de inicio ({start_y}) y no exceder {default_end_year}.")
            except ValueError:
                print("Entrada inválida. Por favor, introduce un número.")

        # Obtener trimestre de fin
        while True:
            try:
                eq_input = input(f"Trimestre de fin (1-4) [Default: {default_end_quarter}]: ")
                end_q = int(eq_input) if eq_input else default_end_quarter
                
                # Validar rango
                if 1 <= end_q <= 4 and not (start_y == end_y and start_q > end_q):
                    break
                else:
                    print(f"El trimestre debe ser un número entre 1 y 4, y no puede ser anterior al trimestre de inicio si es el mismo año.")
            except ValueError:
                print("Entrada inválida. Por favor, introduce un número.")

        return start_y, start_q, end_y, end_q

    # 2. Obtener y ejecutar
    s_y, s_q, e_y, e_q = get_user_input()
    print(f"\nComenzando la descarga desde {s_y} T{s_q} hasta {e_y} T{e_q}...")
    
    download_and_extract_data(s_y, s_q, e_y, e_q)

--- Configuración de Rango de Descarga (Formato AAAA T) ---

Comenzando la descarga desde 2023 T1 hasta 2025 T2...

⬇️ Intentando descargar enoe_2023_trim1_dbf.zip (1/3)...
✅ Éxito: enoe_2023_trim1_dbf.zip descomprimido en data/ENOE\ENOE20231

⬇️ Intentando descargar enoe_2023_trim2_dbf.zip (1/3)...
✅ Éxito: enoe_2023_trim2_dbf.zip descomprimido en data/ENOE\ENOE20232

⬇️ Intentando descargar enoe_2023_trim3_dbf.zip (1/3)...
✅ Éxito: enoe_2023_trim3_dbf.zip descomprimido en data/ENOE\ENOE20233

⬇️ Intentando descargar enoe_2023_trim4_dbf.zip (1/3)...
✅ Éxito: enoe_2023_trim4_dbf.zip descomprimido en data/ENOE\ENOE20234

⬇️ Intentando descargar enoe_2024_trim1_dbf.zip (1/3)...
✅ Éxito: enoe_2024_trim1_dbf.zip descomprimido en data/ENOE\ENOE20241

⬇️ Intentando descargar enoe_2024_trim2_dbf.zip (1/3)...
✅ Éxito: enoe_2024_trim2_dbf.zip descomprimido en data/ENOE\ENOE20242

⬇️ Intentando descargar enoe_2024_trim3_dbf.zip (1/3)...
✅ Éxito: enoe_2024_trim3_dbf.zip descomprimido en data/ENOE

In [ ]:
# Homologar     

In [ ]:
# Estadística Exploratoria
